# Model monitoring dashboard example

This notebook currently requires the plotly library to be installed.


In [ ]:
import weave
import weave.legacy.weave.panels
weave.use_lazy_execution()
from weave.legacy.scripts import syndata_mon

## Create synthetic data


In [ ]:
data = syndata_mon.random_predictions(10)
timestamps = data._arrow_data.field('timestamp')
start_date = timestamps[0].as_py().to_pydatetime()
end_date = timestamps[-1].as_py().to_pydatetime()
predictions = weave.save(data, 'predictions')

## Create a Weave Board for the data


In [ ]:
# Fetch the data by the latest alias instead of its version hash.
# TODO: change weave.save to do this automatically
predictions = weave.legacy.weave.ops.get("local-artifact:///predictions:latest/obj")

In [ ]:
# from weave.legacy.weave import context_state
# context_state._eager_mode.get()

In [ ]:
# Create a dashboard for our data
board = weave.legacy.weave.panels.Board(
    vars={
        'data': predictions,
        'zoom_range': weave.legacy.weave.ops.make_list(
            a=weave.legacy.weave.ops.date_parse(str(start_date)),
            b=weave.legacy.weave.ops.date_parse(str(end_date))),
        'data_range': lambda data: weave.legacy.weave.ops.make_list(
            a=data['timestamp'].min(),
            b=data['timestamp'].max()
        ),
        'bin_range': lambda zoom_range, data_range: zoom_range.coalesce(data_range),
        'predictions_query': weave.legacy.weave.panels.Query(
            predictions,
            conditions=[
                lambda query_input: weave.legacy.weave.panels.QueryCondition(
                    expression=query_input['model_version'],
                    editor=weave.legacy.weave.panels.SelectEditor(
                        choices=query_input['model_version']
                    )
                ),
                lambda query_input: weave.legacy.weave.panels.QueryCondition(
                    expression=query_input['username'],
                    editor=weave.legacy.weave.panels.SelectEditor(
                        choices=query_input['username']
                    )
                ),
            ]),
        'predictions': lambda predictions_query: predictions_query.selected()
    },
    panels=[
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, zoom_range, bin_range: weave.legacy.weave.panels.Plot(
                            predictions,
                            x=lambda item: item["timestamp"].bin(
                                weave.legacy.weave.ops.timestamp_bins_nice(bin_range, 30)),
                            label=lambda item: item["model_version"],
                            y=lambda preds: preds.count(),
                            groupby_dims=['x', 'label'],
                            domain_x=zoom_range,
                            mark="bar",
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=12, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, zoom_range, bin_range: weave.legacy.weave.panels.Plot(
                            predictions,
                            x=lambda item: item["timestamp"].bin(
                                weave.legacy.weave.ops.timestamp_bins_nice(bin_range, 100)),
                            label=lambda item: item["username"],
                            y=lambda preds: preds.count(),
                            groupby_dims=['x', 'label'],
                            domain_x=zoom_range,
                            mark="bar",
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=12, y=0, w=12, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, zoom_range, bin_range: weave.legacy.weave.panels.Plot(
                            predictions,
                            x=lambda item: item["timestamp"].bin(
                                weave.legacy.weave.ops.timestamp_bins_nice(bin_range, 100))['start'],
                            y=lambda preds: preds["api_cost"].sum(),
                            groupby_dims=['x'],
                            domain_x=zoom_range,
                            mark="line",
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=6, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, zoom_range, bin_range: weave.legacy.weave.panels.Plot(
                            predictions,
                            x=lambda item: item["timestamp"].bin(
                                weave.legacy.weave.ops.timestamp_bins_nice(bin_range, 100))['start'],
                            y=lambda preds: preds["prompt_tokens"].sum() + preds['completion_tokens'].sum(),
                            groupby_dims=['x'],
                            domain_x=zoom_range,
                            mark="line",
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=8, y=6, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, zoom_range, bin_range: weave.legacy.weave.panels.Plot(
                            predictions,
                            x=lambda item: item["timestamp"].bin(
                                weave.legacy.weave.ops.timestamp_bins_nice(bin_range, 100))['start'],
                            y=lambda preds: preds["latency"].avg(),
                            groupby_dims=['x'],
                            domain_x=zoom_range,
                            mark="line",
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=16, y=6, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions: weave.legacy.weave.panels.Plot(
                            predictions,
                            x=lambda item: item["api_cost"].bin(
                                weave.legacy.weave.ops.numbers_bins_equal(predictions['api_cost'], 50)),
                            y=lambda preds: preds.count(),
                            groupby_dims=['x'],
                            mark="bar",
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=12, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions: weave.legacy.weave.panels.Plot(
                            predictions,
                            x=lambda item: (item['prompt_tokens'] + item['completion_tokens']).bin(
                                weave.legacy.weave.ops.number_bins_fixed(25)),
                            y=lambda preds: preds.count(),
                            groupby_dims=['x'],
                            mark="bar",
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=8, y=12, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions: weave.legacy.weave.panels.Plot(
                            predictions,
                            x=lambda item: item['latency'].bin(
                                weave.legacy.weave.ops.numbers_bins_equal(predictions['latency'], 50)),
                            y=lambda preds: preds.count(),
                            groupby_dims=['x'],
                            mark="bar",
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=16, y=12, w=8, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions: predictions,
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=18, w=24, h=12)
        )
    ]
)

In [ ]:
board